In [1]:
import multiprocessing as mp

# multiprocessing — Process-based parallelism

https://docs.python.org/3/library/multiprocessing.html

_Depending on the platform, multiprocessing supports three ways to start a process. These start methods are_


__spawn__:

The parent process starts a fresh python interpreter process. The child process will only inherit those resources necessary to run the process object’s run() method. In particular, unnecessary file descriptors and handles from the parent process will not be inherited. Starting a process using this method is rather slow compared to using fork or forkserver.

Available on Unix and Windows. The default on Windows (always) and macOS (since python 3.8+).

__fork__:

The parent process uses `os.fork()` to fork the Python interpreter. The child process, when it begins, is effectively identical to the parent process. All resources of the parent are inherited by the child process. Note that safely forking a multithreaded process is problematic.

Available on Unix only. The default on Unix.


__forkserver__:

When the program starts and selects the forkserver start method, a server process is started. From then on, whenever a new process is needed, the parent process connects to the server and requests that it fork a new process. The fork server process is single threaded so it is safe for it to use `os.fork()`. No unnecessary resources are inherited.

Available on Unix platforms which support passing file descriptors over Unix pipes.


## spawn

<img src="http://www.bnikolic.co.uk/blog/assets/2019-11-13-python-forkserver-preload/python-forkserver-1.png" align="left"/>

## fork

<img src="http://www.bnikolic.co.uk/blog/assets/2019-11-13-python-forkserver-preload/python-forkserver-0.png" align="left"/>


## forkserver

<img src="http://www.bnikolic.co.uk/blog/assets/2019-11-13-python-forkserver-preload/python-forkserver-2.png" align="left"/>

[Python forkserver and set_forkserver_preload](http://www.bnikolic.co.uk/blog/python/parallelism/2019/11/13/python-forkserver-preload.html)

## multiprocessing context

In [2]:
import multiprocessing as mp


if __name__ == "__main__":
    mp.set_start_method("fork")

In [3]:
import multiprocessing as mp


if __name__ == "__main__":
    mp.set_start_method("fork")

RuntimeError: context has already been set

In [4]:
import multiprocessing as mp


if __name__ == "__main__":
    for methods in ("spawn", "fork", "forkserver"):
        mp_context = mp.get_context(methods)
        print(f"{methods!r}: {mp_context}: f{dir(mp_context)}\n\n")
    
    print(f"mp: {mp}: f{dir(mp_context)}\n\n")

'spawn': <multiprocessing.context.SpawnContext object at 0x7efda80f7820>: f['Array', 'AuthenticationError', 'Barrier', 'BoundedSemaphore', 'BufferTooShort', 'Condition', 'Event', 'JoinableQueue', 'Lock', 'Manager', 'Pipe', 'Pool', 'Process', 'ProcessError', 'Queue', 'RLock', 'RawArray', 'RawValue', 'Semaphore', 'SimpleQueue', 'TimeoutError', 'Value', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_available', '_name', 'active_children', 'allow_connection_pickling', 'cpu_count', 'current_process', 'freeze_support', 'get_context', 'get_logger', 'get_start_method', 'log_to_stderr', 'parent_process', 'reducer', 'set_executable', 'set_forkserver_preload', 'set_start_method']


'fork': <multip

## Process Objects

`multiprocessing.Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)`

In [5]:
help(mp.Process)

Help on class Process in module multiprocessing.context:

class Process(multiprocessing.process.BaseProcess)
 |  Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |  
 |  Process objects represent activity that is run in a separate process
 |  
 |  The class is analogous to `threading.Thread`
 |  
 |  Method resolution order:
 |      Process
 |      multiprocessing.process.BaseProcess
 |      builtins.object
 |  
 |  Methods inherited from multiprocessing.process.BaseProcess:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  close(self)
 |      Close the Process object.
 |      
 |      This method releases resources held by the Process object.  It is
 |      an error to call this method if the child process is still running.
 |  
 |  is_alive(self)
 |      Return whether process i

In [1]:
import threading
import os
from functools import wraps
import time

def thread_info(fn):

    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"{threading.current_thread().name}. Start function: {fn.__name__}. {args = }, {kwargs = }")
        try:
            return fn(*args, **kwargs)
        finally:
            print(f"{threading.current_thread().name}. End function: {fn.__name__}")

    return wrapper


def process_info(fn):

    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"Process ID: {os.getpid()}. Parrent Process: {os.getppid()}. {__name__ = }. Start function: {fn.__name__}. {args = }, {kwargs = }")
        try:
            return fn(*args, **kwargs)
        finally:
            print(f"Process ID: {os.getpid()}. End function: {fn.__name__}")

    return wrapper

In [9]:
import multiprocessing as mp
import os

IMMUTABLE = 1
MUTABLE = ["1", "2", "3", "4"]

@process_info
@thread_info
def function_run_in_mp(arg1, arg2, kwarg1=None, kwarg2=None):
    print(f"{arg1 = }, {arg2 = }, {kwarg1 = }, {kwarg2 = }")
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    time.sleep(4)
    
    IMMUTABLE += 1
    MUTABLE[0] = ["value"]
    
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    print("Thats All Folks")

    
if __name__ == "__main__":   # Mandatory
    print(f"Execute application. PID: {os.getpid()}")
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    
    pr = mp.Process(target=function_run_in_mp, args=(1, 2), kwargs={"kwarg1": 3, "kwarg2": 4})
    pr.start()
    pr.join()
    
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    print(f"Complete. PID: {os.getpid()}")

Execute application. PID: 12132
PID: 12132. IMMUTABLE = 1 9784864, MUTABLE = ['1', '2', '3', '4'] 139627910972992
Process ID: 14770. Parrent Process: 12132. __name__ = '__main__'. Start function: function_run_in_mp. args = (1, 2), kwargs = {'kwarg1': 3, 'kwarg2': 4}
MainThread. Start function: function_run_in_mp. args = (1, 2), kwargs = {'kwarg1': 3, 'kwarg2': 4}
arg1 = 1, arg2 = 2, kwarg1 = 3, kwarg2 = 4
MainThread. End function: function_run_in_mp
Process ID: 14770. End function: function_run_in_mp


Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-eeaff565bd70>", line 25, in wrapper
    return fn(*args, **kwargs)
  File "<ipython-input-8-eeaff565bd70>", line 12, in wrapper
    return fn(*args, **kwargs)
  File "<ipython-input-9-94e5b2d9f819>", line 11, in function_run_in_mp
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
UnboundLocalError: local variable 'IMMUTABLE' referenced before assignment


PID: 12132. IMMUTABLE = 1 9784864, MUTABLE = ['1', '2', '3', '4'] 139627910972992
Complete. PID: 12132


In [12]:
import multiprocessing as mp
import os

IMMUTABLE = 1
MUTABLE = ["1", "2", "3", "4"]

@process_info
@thread_info
def function_run_in_mp(arg1, arg2, kwarg1=None, kwarg2=None):
    print(f"{arg1 = }, {arg2 = }, {kwarg1 = }, {kwarg2 = }")
    
    IMMUTABLE = arg1
    MUTABLE = arg2
    
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    time.sleep(4)
    
    IMMUTABLE += 1
    MUTABLE[0] = "value"
    
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    print("Thats All Folks")

    
if __name__ == "__main__":
    
    print(f"Execute application. PID: {os.getpid()}")
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    
    pr = mp.Process(target=function_run_in_mp, args=(IMMUTABLE, MUTABLE))
    pr.start()
    pr.join()
    
    print(f"PID: {os.getpid()}. {IMMUTABLE = } {id(IMMUTABLE)}, {MUTABLE = } {id(MUTABLE)}")
    print(f"Complete. PID: {os.getpid()}")

Execute application. PID: 12132
PID: 12132. IMMUTABLE = 1 9784864, MUTABLE = ['1', '2', '3', '4'] 139627910915840
Process ID: 14896. Parrent Process: 12132. __name__ = '__main__'. Start function: function_run_in_mp. args = (1, ['1', '2', '3', '4']), kwargs = {}
MainThread. Start function: function_run_in_mp. args = (1, ['1', '2', '3', '4']), kwargs = {}
arg1 = 1, arg2 = ['1', '2', '3', '4'], kwarg1 = None, kwarg2 = None
PID: 14896. IMMUTABLE = 1 9784864, MUTABLE = ['1', '2', '3', '4'] 139627910915840
PID: 14896. IMMUTABLE = 2 9784896, MUTABLE = ['value', '2', '3', '4'] 139627910915840
Thats All Folks
MainThread. End function: function_run_in_mp
Process ID: 14896. End function: function_run_in_mp
PID: 12132. IMMUTABLE = 1 9784864, MUTABLE = ['1', '2', '3', '4'] 139627910915840
Complete. PID: 12132


In [13]:
import multiprocessing as mp


@process_info
@thread_info
def function_run_in_mp(arg1, arg2, kwarg1=None, kwarg2=None):
    print(f"{arg1 = }, {arg2 = }, {kwarg1 = }, {kwarg2 = }")
    time.sleep(4)
    print("Thats All Folks")

    
if __name__ == "__main__":
    import os
    print(f"Execute application. PID: {os.getpid()}")
    
    context = mp.get_context("fork")
    
    # Create from mp context
    pr = context.Process(target=function_run_in_mp, args=(1, 2), kwargs={"kwarg1": 3, "kwarg2": 4})
    pr.start()
    pr.join()
    
    print(f"Complete. PID: {os.getpid()}")

Execute application. PID: 12132
Process ID: 14935. Parrent Process: 12132. __name__ = '__main__'. Start function: function_run_in_mp. args = (1, 2), kwargs = {'kwarg1': 3, 'kwarg2': 4}
MainThread. Start function: function_run_in_mp. args = (1, 2), kwargs = {'kwarg1': 3, 'kwarg2': 4}
arg1 = 1, arg2 = 2, kwarg1 = 3, kwarg2 = 4
Thats All Folks
MainThread. End function: function_run_in_mp
Process ID: 14935. End function: function_run_in_mp
Complete. PID: 12132


In [16]:
import multiprocessing as mp


@process_info
@thread_info
def function_run_in_mp(arg1, arg2, kwarg1=None, kwarg2=None):
    print(f"{arg1 = }, {arg2 = }, {kwarg1 = }, {kwarg2 = }")
    time.sleep(4)
    print("Thats All Folks")

    
if __name__ == "__main__":
    import os
    print(f"Execute application. PID: {os.getpid()}")
    
    processes = []
    for _ in range(4):
        pr = mp.Process(target=function_run_in_mp, args=(1, 2), kwargs={"kwarg1": 3, "kwarg2": 4})
        pr.start()
        processes.append(pr)
        
    for pr in processes:
        pr.join()
    
    print(f"Complete. PID: {os.getpid()}")

In [1]:
import multiprocessing as mp
import queue

attr_queue = dir(queue.Queue())
attr_mp_queue = dir(mp.Queue())
attr_mp_joinable_queue = dir(mp.JoinableQueue())

print(f"{mp.Queue = }, diff with queue.Queue {', '.join(x for x in attr_queue if x not in attr_mp_queue)}")
print()
print(f"{mp.JoinableQueue = }, diff with queue.Queue {', '.join(x for x in attr_queue if x not in attr_mp_joinable_queue)}")

mp.Queue = <bound method BaseContext.Queue of <multiprocessing.context.DefaultContext object at 0x7f99084cda90>>, diff with queue.Queue _get, _init, _put, _qsize, all_tasks_done, join, maxsize, mutex, not_empty, not_full, queue, task_done, unfinished_tasks

mp.JoinableQueue = <bound method BaseContext.JoinableQueue of <multiprocessing.context.DefaultContext object at 0x7f99084cda90>>, diff with queue.Queue _get, _init, _put, _qsize, all_tasks_done, maxsize, mutex, not_empty, not_full, queue, unfinished_tasks


## pickle — Python object serialization

https://docs.python.org/3/library/pickle.html

In [4]:
import pickle

d = dict(zip(range(10), range(10)))
d_pickled = pickle.dumps(d)
d1 = pickle.loads(d_pickled)

print(d, id(d), d1, id(d1))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9} 140294870281728 {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9} 140294870450624


In [5]:
d_pickled

b'\x80\x04\x95-\x00\x00\x00\x00\x00\x00\x00}\x94(K\x00K\x00K\x01K\x01K\x02K\x02K\x03K\x03K\x04K\x04K\x05K\x05K\x06K\x06K\x07K\x07K\x08K\x08K\tK\tu.'

Basic work with mp.queues (Pseudo Code)

1. o = pickle.dump(input_object)
2. o -> put_into_queue()
3. raw = get_from_queue() -> o
4. output_object = pickle.load(raw)

In [8]:
from queue import Empty
import multiprocessing as mp


storage = mp.Queue()

@process_info
@thread_info
def add_to_storage(x, value={}):
    for i in range(x):
        value[i] = i
        print(f"Put: {value = } {id(value)}")
        
        storage.put(value)
        time.sleep(0.0001)  # Let's simulate I/O operation

@process_info
@thread_info
def read_from_storage():
    while True:
        try:
            value = storage.get(timeout=1)
        except Empty:
            break
               
        print(f"Get: {value = } {id(value)}")
        value["origin"] = "new_value"
        
            
if __name__ == "__main__":
    print(f"Execute application. PID: {os.getpid()}")
    
    MUTTABLE = {"origin": "value"}
    print(f"Origin: {MUTTABLE = } {id(MUTTABLE)}")
    
    
    write_worker = mp.Process(target=add_to_storage, name="Writer-Process", args=(5, MUTTABLE, ))
    read_worker = mp.Process(target=read_from_storage, name="Reader-Process")
        
    write_worker.start()
    read_worker.start()
        
    write_worker.join()
    read_worker.join()
    
    print(f"Origin: {MUTTABLE = } {id(MUTTABLE)}")
    print(f"Complete. PID: {os.getpid()}")

Execute application. PID: 17684
Origin: MUTTABLE = {'origin': 'value'} 140294867034944
Process ID: 17771. Parrent Process: 17684. __name__ = '__main__'. Start function: add_to_storage. args = (5, {'origin': 'value'}), kwargs = {}
MainThread. Start function: add_to_storage. args = (5, {'origin': 'value'}), kwargs = {}Process ID: 17774. Parrent Process: 17684. __name__ = '__main__'. Start function: read_from_storage. args = (), kwargs = {}

Put: value = {'origin': 'value', 0: 0} 140294867034944MainThread. Start function: read_from_storage. args = (), kwargs = {}

Get: value = {'origin': 'value', 0: 0} 140294866929152Put: value = {'origin': 'value', 0: 0, 1: 1} 140294867034944

Get: value = {'origin': 'value', 0: 0, 1: 1} 140294866930560Put: value = {'origin': 'value', 0: 0, 1: 1, 2: 2} 140294867034944

Get: value = {'origin': 'value', 0: 0, 1: 1, 2: 2} 140294869845120Put: value = {'origin': 'value', 0: 0, 1: 1, 2: 2, 3: 3} 140294867034944

Put: value = {'origin': 'value', 0: 0, 1: 1, 2: 

## `Pipe(duplex=True)`

In [9]:
help(mp.Pipe)

Help on method Pipe in module multiprocessing.context:

Pipe(duplex=True) method of multiprocessing.context.DefaultContext instance
    Returns two connection object connected by a pipe



In [11]:
import multiprocessing as mp
import os


@process_info
@thread_info
def pipe_sample(pipe_connection):
    pipe_connection.send(42)
    pipe_connection.send(None)
    pipe_connection.send(True)
    pipe_connection.close()

if __name__ == '__main__':
    print(f"Execute application. PID: {os.getpid()}\n\n")
    
    
    parent_conn, child_conn = mp.Pipe()
    
    pr = mp.Process(target=pipe_sample, args=(child_conn,))
    pr.start()
    
    for _ in range(3):
        print(f"{parent_conn = }: {parent_conn.recv()}")
    
    pr.join()
    
    print(f"\n\nComplete. PID: {os.getpid()}")

Execute application. PID: 17684


Process ID: 17859. Parrent Process: 17684. __name__ = '__main__'. Start function: pipe_sample. args = (<multiprocessing.connection.Connection object at 0x7f98f1b8cdc0>,), kwargs = {}
MainThread. Start function: pipe_sample. args = (<multiprocessing.connection.Connection object at 0x7f98f1b8cdc0>,), kwargs = {}
MainThread. End function: pipe_sample
Process ID: 17859. End function: pipe_sample
parent_conn = <multiprocessing.connection.Connection object at 0x7f98f1ed8490>: 42
parent_conn = <multiprocessing.connection.Connection object at 0x7f98f1ed8490>: None
parent_conn = <multiprocessing.connection.Connection object at 0x7f98f1ed8490>: True


Complete. PID: 17684


In [15]:
import multiprocessing as mp
import os


@process_info
@thread_info
def pipe_sample(pipe_connection):
    pipe_connection.send(42)
    pipe_connection.send(True)
    pipe_connection.send(None)
    pipe_connection.close()

if __name__ == '__main__':
    print(f"Execute application. PID: {os.getpid()}\n\n")
    
    
    parent_conn, child_conn = mp.Pipe()
        
    pr = mp.Process(target=pipe_sample, args=(child_conn,))
    pr.start()
    
    while True:
        value = parent_conn.recv()
        if value is None:
            break
        print(f"{parent_conn = }: {value = }")
    
    pr.join()
    
    print(f"\n\nComplete. PID: {os.getpid()}")

Execute application. PID: 17684


Process ID: 17935. Parrent Process: 17684. __name__ = '__main__'. Start function: pipe_sample. args = (<multiprocessing.connection.Connection object at 0x7f98f1ed8700>,), kwargs = {}
MainThread. Start function: pipe_sample. args = (<multiprocessing.connection.Connection object at 0x7f98f1ed8700>,), kwargs = {}
MainThread. End function: pipe_sample
Process ID: 17935. End function: pipe_sample
parent_conn = <multiprocessing.connection.Connection object at 0x7f98f1b6f880>: value = 42
parent_conn = <multiprocessing.connection.Connection object at 0x7f98f1b6f880>: value = True


Complete. PID: 17684


In [17]:
help(mp.Pool)

Help on method Pool in module multiprocessing.context:

Pool(processes=None, initializer=None, initargs=(), maxtasksperchild=None) method of multiprocessing.context.DefaultContext instance
    Returns a process pool object



## `Pool(processes=None, initializer=None, initargs=(), maxtasksperchild=None)`

### `.apply(func[, args[, kwds]])`

In [22]:
import multiprocessing as mp


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=2) as pool:
        result = pool.apply(squares, args=(1, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        result = pool.apply(squares, args=(2, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        result = pool.apply(squares, args=(3, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")

Process ID: 18118. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}
MainThread. Start function: squares. args = (1,), kwargs = {}
MainThread. End function: squares
Process ID: 18118. End function: squares
Process ID: 18119. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}
MainThread. Start function: squares. args = (2,), kwargs = {}
MainThread. End function: squares
Process ID: 18119. End function: squares
Process ID: 18118. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (3,), kwargs = {}
MainThread. Start function: squares. args = (3,), kwargs = {}
MainThread. End function: squares
Process ID: 18118. End function: squares


Complete. PID: 17684. Result = 1, <class 'int'>


Complete. PID: 17684. Result = 4, <class 'int'>


Complete. PID: 17684. Result = 9, <class 'int'>


### `.map(func, iterable[, chunksize])`

In [23]:
import multiprocessing as mp


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.map(squares, range(10))
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")

Process ID: 18188. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (0,), kwargs = {}Process ID: 18189. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}Process ID: 18190. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}


MainThread. Start function: squares. args = (0,), kwargs = {}MainThread. Start function: squares. args = (1,), kwargs = {}MainThread. Start function: squares. args = (2,), kwargs = {}


MainThread. End function: squaresMainThread. End function: squares

Process ID: 18190. End function: squaresProcess ID: 18189. End function: squares

Process ID: 18190. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (3,), kwargs = {}Process ID: 18189. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (4,), kwargs = {}
MainThread. End function: squares

MainThread. Start function: squares. args = (4,), 

### `.imap(func, iterable[, chunksize])`

In [28]:
import multiprocessing as mp


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.imap(squares, range(10))  # Lazy
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        
        for x in result:
            print(x)

Process ID: 18367. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}Process ID: 18368. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}
Process ID: 18366. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (0,), kwargs = {}
MainThread. Start function: squares. args = (1,), kwargs = {}MainThread. Start function: squares. args = (2,), kwargs = {}


MainThread. End function: squaresMainThread. End function: squares
MainThread. Start function: squares. args = (0,), kwargs = {}
Process ID: 18368. End function: squares

Process ID: 18367. End function: squares
MainThread. End function: squares
Process ID: 18367. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (3,), kwargs = {}Process ID: 18368. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (4,), kwargs = {}
Process ID: 18366. End function: squares

MainThrea

In [26]:
print(dir(result))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cache', '_cond', '_index', '_items', '_job', '_length', '_pool', '_set', '_set_length', '_unsorted', 'next']


### `.imap_unordered(func, iterable[, chunksize])`

In [30]:
import multiprocessing as mp


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.imap_unordered(squares, range(10))  # Doesn't guaranties order
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        for x in result:
            print(x)

Process ID: 18559. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (0,), kwargs = {}Process ID: 18560. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}Process ID: 18561. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}


MainThread. Start function: squares. args = (0,), kwargs = {}MainThread. Start function: squares. args = (1,), kwargs = {}MainThread. Start function: squares. args = (2,), kwargs = {}

MainThread. End function: squaresMainThread. End function: squares

Process ID: 18561. End function: squaresProcess ID: 18559. End function: squares

Process ID: 18561. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (3,), kwargs = {}Process ID: 18559. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (4,), kwargs = {}
MainThread. Start function: squares. args = (3,), kwargs = {}
MainThread. Start functi

### `.starmap(func, iterable, chunksize=None)`

In [33]:
import multiprocessing as mp
import operator

if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        arguments = list(zip(range(10), range(10)))
        # arguments = zip(range(10), range(10))

        print(f"{arguments = }")
        
        result = pool.starmap(operator.add, arguments)
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        

arguments = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]


Complete. PID: 17684. Result = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18], <class 'list'>


In [34]:
import multiprocessing as mp
import operator

if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
#         arguments = list(zip(range(10), range(10)))
        arguments = zip(range(10), range(10))

        print(f"{arguments = }")
        
        result = pool.starmap(operator.add, arguments)
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        

arguments = <zip object at 0x7f98f1b2dd40>


Complete. PID: 17684. Result = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18], <class 'list'>


### `multiprocessing.pool.AsyncResult`

#### `apply_async(func[, args[, kwds[, callback[, error_callback]]]]`

In [35]:
import multiprocessing as mp


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.apply_async(squares, args=(1, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        result = pool.apply_async(squares, args=(2, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")


Process ID: 18758. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}

Complete. PID: 17684. Result = <multiprocessing.pool.ApplyResult object at 0x7f98f1b6d550>, <class 'multiprocessing.pool.ApplyResult'>


Complete. PID: 17684. Result = <multiprocessing.pool.ApplyResult object at 0x7f98f1b6dbe0>, <class 'multiprocessing.pool.ApplyResult'>


In [36]:
import multiprocessing as mp


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.apply_async(squares, args=(1, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        print(f"{result.get() = }")
        
        result = pool.apply_async(squares, args=(2, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        print(f"{result.get(timeout=1) = }")

Process ID: 18768. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}
MainThread. Start function: squares. args = (1,), kwargs = {}
MainThread. End function: squares
Process ID: 18768. End function: squares
Process ID: 18769. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}
MainThread. Start function: squares. args = (2,), kwargs = {}
MainThread. End function: squares
Process ID: 18769. End function: squares


Complete. PID: 17684. Result = <multiprocessing.pool.ApplyResult object at 0x7f98f1eb65b0>, <class 'multiprocessing.pool.ApplyResult'>
result.get() = 1


Complete. PID: 17684. Result = <multiprocessing.pool.ApplyResult object at 0x7f98f1eafb50>, <class 'multiprocessing.pool.ApplyResult'>
result.get(timeout=1) = 4


In [37]:
import multiprocessing as mp
import time

@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.apply_async(time.sleep, args=(10, ) )
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        print(f"{result.get(timeout=1) = }")  # Should Raise




Complete. PID: 17684. Result = <multiprocessing.pool.ApplyResult object at 0x7f98f18ad9d0>, <class 'multiprocessing.pool.ApplyResult'>


TimeoutError: 

#### `map_async(func, iterable[, chunksize[, callback[, error_callback]]])`

In [39]:
import multiprocessing as mp
import time

@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.map_async(squares, range(10))
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        print(result.get(timeout=1))

Process ID: 18981. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (0,), kwargs = {}
MainThread. Start function: squares. args = (0,), kwargs = {}
MainThread. End function: squares
Process ID: 18981. End function: squares
Process ID: 18981. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (3,), kwargs = {}
MainThread. Start function: squares. args = (3,), kwargs = {}
MainThread. End function: squares
Process ID: 18981. End function: squares
Process ID: 18981. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (4,), kwargs = {}
MainThread. Start function: squares. args = (4,), kwargs = {}
MainThread. End function: squares
Process ID: 18983. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}Process ID: 18981. End function: squares
MainThread. Start function: squares. args = (2,), kwargs = {}

MainThread. End function: squares
Process ID: 18983. End functi

In [41]:
import multiprocessing as mp
import time

@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.map_async(squares, range(10))  # Doesn't guaranties order
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        while not result.ready():
            print("NOT READY")
            time.sleep(0.2)
        
        result.successful()
        print(result.get(timeout=1))

Process ID: 19319. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (0,), kwargs = {}
MainThread. Start function: squares. args = (0,), kwargs = {}
MainThread. End function: squares
Process ID: 19319. End function: squares
Process ID: 19319. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (3,), kwargs = {}
MainThread. Start function: squares. args = (3,), kwargs = {}
MainThread. End function: squares
Process ID: 19319. End function: squares
Process ID: 19319. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (4,), kwargs = {}
MainThread. Start function: squares. args = (4,), kwargs = {}
Process ID: 19320. Parrent Process: 17684. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}
MainThread. End function: squares
MainThread. Start function: squares. args = (1,), kwargs = {}
Process ID: 19319. End function: squaresProcess ID: 19321. Parrent Process: 17684. __name__ = '__main__'

#### `starmap_async(func, iterable[, chunksize[, callback[, error_callback]]])`

In [43]:
import multiprocessing as mp
import operator


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        arguments = list(zip(range(10), range(10)))
        # arguments = zip(range(10), range(10))

        print(f"{arguments = }")
        
        result = pool.starmap_async(operator.add, arguments)
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        while not result.ready():
            time.sleep(0.2)
        
        result.successful()
        print(result.get(timeout=1))

arguments = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]


Complete. PID: 17684. Result = <multiprocessing.pool.MapResult object at 0x7f98f1e316a0>, <class 'multiprocessing.pool.MapResult'>
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
